# Check Library Version

In [1]:
import warnings
import os
import sys

import numpy as np

import cv2

import keras
import tensorflow as tf

warnings.filterwarnings('ignore')

print('python :', sys.version)
print('')

print('numpy :', np.__version__)
print('')

print('opencv :', cv2.__version__)
print('')

print('keras :', keras.__version__)
print('tf :', tf.__version__)

python : 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]

numpy : 1.18.5

opencv : 4.3.0

keras : 2.4.3
tf : 2.2.0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18249945504656533173,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18209884039112941726
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3133590732
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1683641330792374829
 physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14379780237256609506
 physical_device_desc: "device: XLA_GPU device"]

# Load Model

In [28]:
from keras.models import model_from_json
from keras.optimizers import Adam
import keras.backend as K

class DTNNmodels:
    
    def __init__(self):
        #초기 모델 및 가중치 경로 설정
        self.ROOT_DIR = os.path.abspath('./')
        self.models_path = os.path.join(self.ROOT_DIR, 'models')
        self.weights_path = os.path.join(self.models_path, 'weights')

        self.unet_model_file = os.path.join(self.models_path, 'unet1_model.json')
        
        self.unet_weight_skyline_file = os.path.join(self.weights_path, 'skyline_detection_1channel.hdf5')
        self.unet_weight_shielding_file = os.path.join(self.weights_path, 'view_shielding_rate_1channel.hdf5')
        
    def load_models(self):
        #model load
        K.clear_session()
        self.sky_unet = self.__unet_model(self.unet_model_file, self.unet_weight_skyline_file)
        self.shield_unet = self.__unet_model(self.unet_model_file, self.unet_weight_shielding_file)
        
        return self.sky_unet, self.shield_unet

    def __unet_model(self, model_file, weight_file):
        #unet model
        K.set_image_data_format('channels_last')
        K.set_floatx('float64')

        json_file = open(model_file, "r")
        loaded_model_json = json_file.read() 
        json_file.close()

        unet = model_from_json(loaded_model_json)
        unet.load_weights(weight_file)

        unet.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics = ['mse', 'mae'])    
        return unet
    
    def predict(self, file_img, model, threshold = 20):
        
        input_img, img_arr = self.__preprocessing_img(file_img)
        
        predictions = model.predict(img_arr)
        prediction = np.array(np.round(predictions[0] * 255, 0), dtype = 'int')
        
        clear_pred = self.__clear_img(prediction, threshold)
        ridge = self.__y_ridge(clear_pred)
        
        resize_img = np.array(input_img.resize((256,256)), np.uint8)
        output_img = self.__draw_skyline(resize_img, ridge)
        rate = self.__shielding_rate(resize_img, ridge)
        
        return output_img, rate
        
    def __preprocessing_img(self, file_img):
        img = Image.open(file_img)
        one_ch_img = img.convert('L')
        input_arr = np.array(one_ch_img.resize((256, 256)))/255
        input_arr = input_arr.reshape((1, ) + input_arr.shape)
        
        return img, input_arr
        
    def __clear_img(self, img_arr, threshold):
        clear = np.where(img_arr < threshold, 0, img_arr)
        clear = np.where(clear > 0, 255, clear)
    
        return clear

    def __y_ridge(self, img_arr):
        cols = img_arr.shape[1]
        rows = img_arr.shape[0]

        ridge = []

        for c in range(cols):
            for r in range(rows):
                row = img_arr[r][c]
                if row > 0:
                    ridge.append([c, r])
                    break

        ridge.append([cols+1, rows+1])
        ridge.append([0, rows+1])            
        ridge_array = np.array(ridge)

        return ridge_array

    def __draw_skyline(self, img, ridge):

        ridge = ridge.astype(np.int32)

        polyline_img = cv2.polylines(img, [ridge], False, (255, 0, 0))

        return polyline_img
    
    def __shielding_rate(self, img, contour):
        x = img.shape[1]
        y = img.shape[0]

        contour_area = cv2.contourArea(contour)
        shield_rate = round((contour_area/(x*y)) * 100, 4)

        return shield_rate

In [30]:
from dtnn.dtnn_model import Predictmodels

ModuleNotFoundError: No module named 'dtnn.dtnn_model'

# Predict Train Image

In [29]:
from flask import Flask, request, jsonify
from PIL import Image, ImageDraw
import json
import base64
from io import BytesIO

app = Flask (__name__)

@app.route('/Dpredict', methods = ['POST'])
def predict():
    if request.method == 'POST':
        
        data = request.form
        img = request.files['image']
        form = data['format']
        command = data['command']
        
        if command == 'skyline_detection':
            
            output_img, _ = dtnn.predict(img, sky_model)
            
            result = {}
            
        elif command == 'view_shielding_rate':
            
            output_img, rate = dtnn.predict(img, shield_model)
        
            result = {'shielding_rate' : rate}
            
        if form == 'total_array':
            output = output_img.tolist()
            result['output_img'] = output
        elif form == 'total_base64':
            rawBytes = BytesIO()
            img_buffer = Image.fromarray(output_img.astype('uint8'))
            img_buffer.save(rawBytes, 'PNG')
            rawBytes.seek(0)
            base64_img = base64.b64encode(rawBytes.read())
            output = str(base64_img)
            result['output_img'] = output
        else :
            print('format을 입력해주세요. 현재 지원 format parameter는 total_array, total_base64입니다.')
            
#        img_result는 output image를 원래의 사이즈로 다시 키우는 것, 화질이 너무 깨져서 다른 방법 생각해야함        
#        img_result = cv2.resize(output_img, dsize = (input_img.size[0], input_img.size[1]), interpolation=cv2.INTER_CUBIC)
    return jsonify({'command' : command, 'result' : result})

In [27]:
if __name__ == "__main__":
    dtnn = DTNNmodels()
    sky_model, shield_model = dtnn.load_models()
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Jul/2020 17:39:30] "POST /Dpredict HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2020 17:39:42] "POST /Dpredict HTTP/1.1" 200 -
